<a href="https://colab.research.google.com/github/PHimajasree/Task2/blob/main/stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import os

class StockPortfolioTracker:
    def __init__(self, api_key):
        self.api_key = api_key
        self.portfolio = {}

    def load_portfolio(self, filename="portfolio.csv"):
        # Load portfolio from a CSV file if it exists
        if os.path.exists(filename):
            df = pd.read_csv(filename)
            self.portfolio = dict(zip(df['Stock'], df['Shares']))
            print("Portfolio loaded successfully.\n")
        else:
            print("No saved portfolio found. Starting fresh.\n")

    def save_portfolio(self, filename="portfolio.csv"):
        # Save portfolio to a CSV file
        df = pd.DataFrame(list(self.portfolio.items()), columns=["Stock", "Shares"])
        df.to_csv(filename, index=False)
        print("Portfolio saved successfully.\n")

    def add_stock(self, symbol, shares):
        # Add or update stock in the portfolio
        if symbol in self.portfolio:
            self.portfolio[symbol] += shares
        else:
            self.portfolio[symbol] = shares
        print(f"Added {shares} shares of {symbol} to your portfolio.\n")

    def remove_stock(self, symbol, shares):
        # Remove or reduce shares of a stock in the portfolio
        if symbol in self.portfolio and self.portfolio[symbol] >= shares:
            self.portfolio[symbol] -= shares
            if self.portfolio[symbol] == 0:
                del self.portfolio[symbol]
            print(f"Removed {shares} shares of {symbol} from your portfolio.\n")
        else:
            print(f"Invalid operation: Not enough shares of {symbol} to remove or stock not in portfolio.\n")

    def get_stock_price(self, symbol):
        # Fetch real-time stock price from Alpha Vantage API
        url = "https://www.alphavantage.co/query"
        params = {
            "function": "TIME_SERIES_INTRADAY",
            "symbol": symbol,
            "interval": "1min",
            "apikey": self.api_key
        }
        response = requests.get(url, params=params)
        data = response.json()

        try:
            latest_time = list(data["Time Series (1min)"].keys())[0]
            price = float(data["Time Series (1min)"][latest_time]["1. open"])
            return price
        except KeyError:
            print(f"Failed to fetch price for {symbol}. Please check the stock symbol or API limits.\n")
            return None

    def track_portfolio(self):
        # Display portfolio details with real-time stock prices
        print("Your Portfolio:")
        total_value = 0
        for symbol, shares in self.portfolio.items():
            price = self.get_stock_price(symbol)
            if price:
                value = price * shares
                total_value += value
                print(f"{symbol}: {shares} shares | Current price: ₹{price} | Value: ₹{value}")
        print(f"Total portfolio value: ₹{total_value}\n")


# Example Usage for Google Colab
if __name__ == "__main__":
    api_key = "YOUR_API_KEY"  # Replace with your Alpha Vantage API key
    tracker = StockPortfolioTracker(api_key)

    # Load an existing portfolio or start a fresh one
    tracker.load_portfolio()

    # Menu for user interaction
    while True:
        print("\nMenu:")
        print("1. Add Stock")
        print("2. Remove Stock")
        print("3. Track Portfolio")
        print("4. Save Portfolio")
        print("5. Exit")

        choice = input("Enter your choice: ")
        if choice == "1":
            symbol = input("Enter stock symbol: ").upper()
            shares = int(input("Enter number of shares: "))
            tracker.add_stock(symbol, shares)
        elif choice == "2":
            symbol = input("Enter stock symbol: ").upper()
            shares = int(input("Enter number of shares to remove: "))
            tracker.remove_stock(symbol, shares)
        elif choice == "3":
            tracker.track_portfolio()
        elif choice == "4":
            tracker.save_portfolio()
        elif choice == "5":
            tracker.save_portfolio()
            print("Exiting the program. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.\n")

No saved portfolio found. Starting fresh.


Menu:
1. Add Stock
2. Remove Stock
3. Track Portfolio
4. Save Portfolio
5. Exit
Added 4 shares of * to your portfolio.


Menu:
1. Add Stock
2. Remove Stock
3. Track Portfolio
4. Save Portfolio
5. Exit
